In [2]:
import numpy as np
import pandas as pd
import os

path = r"/kaggle/input/dhi-club-tired-weekends-4thsem-1/train_data/train_data"
train_set = os.listdir(path)
# print(train_set)

classes = []

for cl in train_set:
    cum = os.listdir(path+"/"+cl)
    for img in cum:
        classes.append((cl, path+"/"+cl+"/"+img))

In [2]:
df = pd.DataFrame(classes, columns = ["Class Labels", "Image path"])
print(df.head())
print(df.tail())

  Class Labels                                         Image path
0       Golbat  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
1       Golbat  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
2       Golbat  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
3       Golbat  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
4       Golbat  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
     Class Labels                                         Image path
8492       Vulpix  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
8493       Vulpix  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
8494       Vulpix  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
8495       Vulpix  /kaggle/input/dhi-club-tired-weekends-4thsem-1...
8496       Vulpix  /kaggle/input/dhi-club-tired-weekends-4thsem-1...


In [3]:
import cv2
IMAGE_SIZE = 224

images = []
labels = []

for cl in train_set:
    lab = os.path.join(path, str(cl))
    img = [i for i in os.listdir(lab)]
    for i in img:
        image = cv2.imread(lab + "/" + i)
        if image is not None:
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            images.append(image)
            labels.append(cl)

In [4]:
images = np.array(images)
images = images.astype("float32")/255.0
images.shape

(8476, 224, 224, 3)

In [5]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()
encoder = OneHotEncoder(sparse_output=False)
labels = np.array(labels)
y = le.fit_transform(labels)
Y = encoder.fit_transform(y.reshape(-1, 1))

In [6]:
x_train = images
y_train = Y
print(x_train.shape)
print(y_train.shape)

(8476, 224, 224, 3)
(8476, 149)


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
augmentor = ImageDataGenerator(zoom_range = 0.3, horizontal_flip = True, width_shift_range=0.2, height_shift_range=0.2, fill_mode="nearest")
datagen = augmentor.flow(x_train, y_train, batch_size = 16)

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 149
RESOLUTION = 224
weights_path = r"/kaggle/input/enetb0/efficientnetb0_notop.h5"
base_model = EfficientNetB0(include_top = False, weights = weights_path, input_shape = (RESOLUTION, RESOLUTION, 3))
base_model.trainable = False
for layer in base_model.layers[:-10]:
  layer.trainable = True
inputs = layers.Input(shape = (RESOLUTION, RESOLUTION, 3))
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation = "relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = "relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(NUM_CLASSES, activation = "softmax")(x)

In [9]:
model = Model(inputs, outputs)
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 149)                 │          38,293 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,883,256 (18.63 MB)

 Trainable params: 3,943,905 (15.04 MB)

 Non-trainable params: 939,351 (3.58 MB)

In [10]:
model.fit(datagen, epochs = 6)
model.save("hopeful.h5")

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


530/530 ━━━━━━━━━━━━━━━━━━━━ 1220s 2s/step - accuracy: 0.0667 - loss: 4.9269
Epoch 2/6
530/530 ━━━━━━━━━━━━━━━━━━━━ 1189s 2s/step - accuracy: 0.2345 - loss: 3.3519
Epoch 3/6
530/530 ━━━━━━━━━━━━━━━━━━━━ 1187s 2s/step - accuracy: 0.3545 - loss: 2.5754
Epoch 4/6
530/530 ━━━━━━━━━━━━━━━━━━━━ 1188s 2s/step - accuracy: 0.4555 - loss: 2.1087
Epoch 5/6
530/530 ━━━━━━━━━━━━━━━━━━━━ 1185s 2s/step - accuracy: 0.5149 - loss: 1.8443
Epoch 6/6
530/530 ━━━━━━━━━━━━━━━━━━━━ 1229s 2s/step - accuracy: 0.5327 - loss: 1.7058


In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

model = tf.keras.models.load_model("/kaggle/input/trained-2/hopeful.h5")
test_path = "/kaggle/input/dhi-club-tired-weekends-4thsem-1/test data/test data"
test_images = os.listdir(test_path)

RESOLUTION = 224
predictions = []

for img_name in test_images:
    img_path = os.path.join(test_path, img_name)
    if img_path[-3:] != "svg":
        test_img = image.load_img(img_path, target_size=(RESOLUTION, RESOLUTION))
        test_img = image.img_to_array(test_img) / 255.0
        test_img = np.expand_dims(test_img, axis=0)
        prediction = model.predict(test_img)
        predicted_class = np.argmax(prediction, axis=1)
        predicted_label = le.inverse_transform(predicted_class)[0]
        predictions.append([img_name, predicted_label])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━

In [13]:
predictions_df = pd.DataFrame(predictions, columns=["img_name", "Class Prediction"])
predictions_df.to_csv("/kaggle/working/hope.csv", index=False)
print("Predictions saved to hope.csv")

Predictions saved to hope.csv
